<a href="https://colab.research.google.com/github/kwakseoyeon/1test/blob/master/building_a_chatbot_app_using_deep_learning_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip gdown

ERROR: unknown command "gdown" - maybe you meant "download"


In [ ]:
import gdown

url = 'https://drive.google.com/uc?id=1ecTFXIaUwAh7I7iLeDK8YCMcPoU4wVwx'
output = 'kogpt_chat_model.pt'  # 원하는 파일 이름과 확장자로 변경
gdown.download(url, output, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1ecTFXIaUwAh7I7iLeDK8YCMcPoU4wVwx
To: /content/kogpt_chat_model.pt
100%|██████████| 501M/501M [00:02<00:00, 168MB/s]


'kogpt_chat_model.pt'

In [ ]:
# 라이브러리 임포트
import torch
from transformers import GPT2Config, GPT2LMHeadModel, PreTrainedTokenizerFast

Q_TKN = "<usr>"
A_TKN = "<sys>"
BOS = '</s>'
EOS = '</s>'
MASK = '<unused0>'
SENT = '<unused1>'
PAD = '<pad>'

koGPT2_TOKENIZER = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
            bos_token=BOS, eos_token=EOS, unk_token='<unk>',
            pad_token=PAD, mask_token=MASK)

# Pretrained 모델의 구성을 로드합니다.
config = GPT2Config.from_pretrained('skt/kogpt2-base-v2')
model = GPT2LMHeadModel(config)
model.load_state_dict(torch.load('kogpt_chat_model.pt'))
model.eval()

sent = "0"

def dl_model(input_text, max_length=40):
    response = ""
    for _ in range(max_length):
        input_ids = torch.LongTensor(koGPT2_TOKENIZER.encode(Q_TKN + input_text + SENT + sent + A_TKN + response)).unsqueeze(dim=0)
        attention_mask = input_ids != koGPT2_TOKENIZER.pad_token_id
        pred = model(input_ids, attention_mask=attention_mask)
        pred = pred.logits
        gen = koGPT2_TOKENIZER.convert_ids_to_tokens(torch.argmax(pred, dim=-1).squeeze().numpy().tolist())[-1]
        if gen == EOS:
            break
        response += gen.replace("▁", " ")

    return response.strip()

tokenizer.json:   0%|          | 0.00/2.83M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [ ]:
def chat_with_user(user_message, chat_history):
    ai_message = dl_model(user_message)
    chat_history.append({'user': user_message, 'ai': ai_message})
    return ai_message

chat_history = []

while True:
    user_message = input("USER > ")
    if user_message.lower() == "quit":
        break
    ai_message = chat_with_user(user_message, chat_history)
    print(f" A I > {ai_message}")

USER > 안녕
 A I > 안녕하세요
USER > 오늘 날씨 어때
 A I > 좋은 아침이에요
USER > 음.. 좋구나. 나도 좋아.
 A I > 좋아요
USER > I'm 신뢰해요.
 A I > 믿기 힘드시겠네요
USER > 좋아. 오늘 저녁 메뉴 추천 부탁해
 A I > 맛있게 드세요
USER > quit
